In [1]:
!pip install transformers torch datasets
!pip install transformers accelerate
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import datasets
from transformers import GPT2TokenizerFast
import torch
from torch.utils.data import DataLoader

In [3]:
from datasets import load_dataset

ds = load_dataset("ESGBERT/environment_data")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


environment_data.csv:   0%|          | 0.00/391M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100586 [00:00<?, ? examples/s]

In [ ]:
ds2 = load_dataset("exo-is/environmental-reporting-on-listed-companies")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

environmental_keywords.csv:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17277 [00:00<?, ? examples/s]

In [4]:
ds['train'][1]['sentence']

'Working with community and Indigenous-led coalitions, we focused on undoing the damage of the Trump years and pursuing public lands solutions to climate change, species extinction and expanding equitable access to nature.'

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 2100586
    })
})

In [ ]:
ds2['train']['Keyword-in-context (750 characters before and after Keyword)'][800]

'Nonhazardous solid waste intensity\t\n0.01\t\n0.01\t\n0.01\nHazardous waste intensity\t\n0.01\t\n0.02\t\n0.03\nPerformance data\nMaterial use\nNonhazardous \nsolid waste by \ndestination\n(MT)\nHazardous solid \nwaste by \ndestination\n(MT)\nWaste intensities \n(MT / MT)\nMaterial use\nThe scope of raw material data is limited to global supply chain \nmanufacturing facilities. Ecolab purchases reconditioned drums and uses \nreusable containers to avoid the use of virgin plastic. The scope of post-\nconsumer resin packaging is North America, Europe and China. The scope \nof reconditioned and reusable packaging is North America and Europe.\nNonhazardous solid waste by destination\nAll waste is disposed of directly by the organization or otherwise directly \nconfirmed by the waste disposal contractor'

In [5]:
# You can limit the number of training samples if needed. Doing so will reduce the quality of your model but will reduce training times.
# I recommend this only when setting up your code (so you don't need to wait for the map function while making things work).
NUM_TRAIN_SAMPLES = len(ds['train'])//20

if NUM_TRAIN_SAMPLES is not None:
  ds['train'] = ds['train'].shuffle(seed=42).select(range(NUM_TRAIN_SAMPLES))

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 105029
    })
})

In [7]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
len(tokenizer)

50257

In [9]:
# We also add a padding token to allow us to create training batches of a fixed size.
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [10]:
# This isn't the only way to tokenize your dataset, but it's a good starting point. The GPT2TokenizerFast class has a lot of useful functions built-in.
# We eventually want to create batches of sequences where each sequence has the same number of tokens (denoted here as "context size"). This tokenization method will ensure that every sequence is the same size.
# truncation=True will truncate any sequences that are longer than the "context size," and padding='max_length' will add PAD tokens to the end of any sequence that is shorter than the "context size."
# Depending on how you do your training, you may need to configure your loss so that it ignores padding tokens.
CONTEXT_SIZE = 512 # This will need to be adjusted based on the context size of your model.
tokenized_dataset = ds.map(lambda examples: tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=CONTEXT_SIZE), batched=True)

Map:   0%|          | 0/105029 [00:00<?, ? examples/s]

In [11]:
# The collate function is where we can preprocess our sequence batches.
# Here, we combine the input sequences into a single batch tensor (and do the same for our attention masks).
def collate_fn(batch):

  input_ids = torch.stack([torch.tensor(item['input_ids']) for item in batch])
  attention_mask = torch.stack([torch.tensor(item['attention_mask']) for item in batch])

  return {
      'input_ids': input_ids,
      'attention_mask': attention_mask
  }

In [12]:
# DataLoaders make it easy to batch our data and use our collate function efficiently. You don't need to use them, but they will likely speed up your training time significantly.
# You can iterate over the dataloader to get batches of (preprocessed) sequences.
train_dataloader = DataLoader(tokenized_dataset['train'], batch_size=4, collate_fn=collate_fn, shuffle=True)
# val_dataloader = DataLoader(tokenized_dataset['validation'], batch_size=4, collate_fn=collate_fn, shuffle=False)

In [20]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

# Customize the GPT-2 model
config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions = CONTEXT_SIZE,
    n_embd = 512,
    n_layer = 2,
    n_head = 8
)

model = GPT2LMHeadModel(config)

# Resize token embeddings in case you're adding custom tokens
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 512)

In [14]:
# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to GPU
model.to(device)

Using device: cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 512)
    (wpe): Embedding(512, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=1536, nx=512)
          (c_proj): Conv1D(nf=512, nx=512)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=2048, nx=512)
          (c_proj): Conv1D(nf=512, nx=2048)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=50258, bias=False)
)

In [ ]:
from accelerate import Accelerator
from tqdm import tqdm  # Import tqdm for progress bars
from torch.optim import AdamW

epochs = 4
optimizer = AdamW(model.parameters(), lr = 1e-4)

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Accelerator
accelerator = Accelerator()
device = accelerator.device # Get the device from Accelerator

# Prepare the model, optimizer, and dataloaders for training with Accelerator
model, optimizer, train_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader
)

# Training loop with GPU usage and gradient accumulation
for epoch in range(epochs):
    total_loss = 0
    model.train()  # Explicitly set model to training mode

    # Wrap the training loop with tqdm for progress bar
    with tqdm(train_dataloader, desc=f"Epoch {epoch+1}", unit="batch") as train_bar:
        for batch in train_bar:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['input_ids'])

            # Compute loss
            loss = outputs.loss
            total_loss += loss.item()

            # Backward pass and optimization
            accelerator.backward(loss)  # Use accelerator.backward
            optimizer.step()

            # Update the progress bar with the current loss
            train_bar.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_dataloader)

    # Print results after the epoch
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss}")


Using device: cuda


Epoch 1: 100%|██████████| 26258/26258 [1:10:01<00:00,  6.25batch/s, loss=0.29]


Epoch 1, Training Loss: 0.3673656065641508


Epoch 2:  23%|██▎       | 6014/26258 [16:11<54:37,  6.18batch/s, loss=0.397]

In [18]:
import os

# Define output directory for saving the model and tokenizer
output_dir = "fine_tuned_gpt2"
os.makedirs(output_dir, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}.")

Model and tokenizer saved to fine_tuned_gpt2.


In [19]:
input_texts = ["Flooding precautions"]

for prompt in input_texts:
  print(f"Prompt: {prompt}")
  for num_beam in [1,5,10]:
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate text using beam search
    beam_output = model.generate(input_ids,
                                num_beams=num_beam,
                                no_repeat_ngram_size=2,
                                max_length=300,
                                early_stopping=True)

    generated_text = tokenizer.decode(beam_output[0], skip_special_tokens=True)
    print(f"Number of beams: {num_beam}")
    print("Generated Text: ")
    print(generated_text)
    print('-' * 50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: Flooding precautions


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Number of beams: 1
Generated Text: 
Flooding precautions to reduce the environmental impact of our products and the environment. . and we operate. . .com/en/or/environmental.com.pdf/p.aspx https://www.org/doc/t/diversity/responsibility/files/pdf https/Pages/sustainability/and/about-environment.html https.5%20%2%% https http://d.net/us/a.3%/social/media/c.2/v.7/corporate/h/cscc/2021/index.g.6%pdf – and/www, and environmental impacts.youtube/re.v/ httpswww., https https-t) and social and other issues.cd/climate/watch/2020.4% httpwww-responsage/com% .pdf to the Group.gov/r.S.c% reduction in our operations.pustain.au/default.ca/investment of the company.d/1% 1.0%D.1/content/b.8% of climate change.ON%respons.9%1.e.t.A.edu/2m%5.bc., and our business.00%7 httpsporate and safety, we have of stakeholders.co.r., we are a positive impact.04% we can have a result to our stakeholders .org% to
--------------------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Number of beams: 5
Generated Text: 
Flooding precautions to the environment, and the communities where we operate and we work to ensure that our employees operate in a safe and healthy environment is responsible for our communities and communities are in which they operate, we have a positive impact on our business and our operations, as well as our ability to operate our customers, our stakeholders and their communities, which we do our planet, in our supply chain, results of operations and results, including our products and suppliers, communities. to our people and business, partners, customers and society.com and other communities that we serve, suppliers and customers are committed to meet our clients and partners and employees, the planet.’s business partners to make our suppliers to manage our value chain and resources and contractors, while our industry and to create a right to develop our environment. . and people, where they have the right. – our global communities in the wor

# Trying out the RAG System

In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model_path = "fine_tuned_gpt2"  # Replace with the actual path to your saved model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# context = 'Water, food and agriculture are the sectors most frequently cited by developing countries as key  sectors for investment in both capacity-building and technology transfer for addressing climate  change adaptation.Yet considering the pace  of climate change, a boost in support of adaptation  implementation is urgently needed Greater focus on and  support provided for adaptation in the next round of NDCs  could give credence to strengthened country ambitions  and actions. Benefits for other types of strategies might be linked to economic development,  public health, air and water quality, and improved housing In some situations, climate adaptation  strategies could also contribute to reductions in GHG emissions. Put simply, those with  the fewest resources to cope with climate change are often the most vulnerable to its impacts  Keeping equity at the center of climate adaptation planning is key to ensuring the entire  community – not just a small subset – has access to the resources needed to adapt to climate  change. Countries need to ramp up  their ambitions to prepare for increasing climate risks Across different data sources, information on the  implementation of adaptation actions shows large annual  fluctuations but they ultimately result in a slight upward  trend over time (figure ES.2)'
context = "Water, food and agriculture are the sectors most frequently cited by developing countries as key  sectors for investment in both capacity-building and technology transfer for addressing climate  change adaptation"
# Generate text
input_ids = tokenizer.encode(context, return_tensors="pt").to(device)
beam_output = model.generate(input_ids,
                            num_beams=10,
                            no_repeat_ngram_size=2,
                            max_length=300,
                            early_stopping=True)

generated_text = tokenizer.decode(beam_output[0], skip_special_tokens=True)
generated_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Using device: cuda


'Water, food and agriculture are the sectors most frequently cited by developing countries as key  sectors for investment in both capacity-building and technology transfer for addressing climate  change adaptation, and climate-related risks, as well as in the supply chain, such as energy, water and water use, waste management, energy and energy efficiency, climate change and greenhouse gas (GHG) emissions, emissions and other risks associated with the impact of our operations, in our products and the world’s ability to our business, we operate, including our customers, which we have a result in order to reduce our own operations and environmental impact on our ability of the environment, while we continue to meet our suppliers, our stakeholders”) and our global operations are more sustainable development and we use of new products, the communities and resources, by our clients and society and emissions reduction of water-saving products we are a global supply chains, to operate in line

In [4]:
len(context), len(generated_text)

(212, 1571)